<a href="https://colab.research.google.com/github/present42/PyTorchPractice/blob/main/Fluent_Python_ch11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 11. A Pythonic Object

Coverage of this chapter:
 * Support the built-in fcns that convert objects to other types
 * Implement an alternative contstructor as a class method
 * extend the format mini-language used by f-strings
 * Provide read-only access to attributes
 * make an object hashable for use in sets and as dict keys
 * save memory with the use of `__slot__`

In [ ]:
# vector2d_v0.py
from array import array
import math

class Vector2d:
  typecode = 'd' # typecode is a class attr
                 # we'll use when converting Vector2d <=> bytes

  def __init__(self, x, y):
    self.x = float(x) # converting x to float catches errors early
    self.y = float(y)

  def __iter__(self):
    """
    __iter__ makes a Vector2d iterable
    """
    # equivalent to yield self.x; yield self.y
    return (i for i in (self.x, self.y))

  def __repr__(self):
    """
    __repr__ builds a string by interpolating the components
    with {!r} to get their repr; because Vector2d is iterable
    *self feeds the x and y component to format
    """
    class_name = type(self).__name__
    return '{}({!r}, {!r})'.format(class_name, *self)

  def __str__(self):
    return str(tuple(self))

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) +
            bytes(array(self.typecode, self)))

  def __eq__(self, other):
    """
    To quickly compare all components,
    build tuples out of the operands
    But there is a side effect:
    e.g. Vector(3, 4) == [3, 4]

    """
    return tuple(self) == tuple(other)

  def __abs__(self):
    return math.hypot(self.x, self.y)

  def __bool__(self):
    return bool(abs(self))

But we still need a way to rebuild a Vector2d from the binary representation produced by bytes()

In [ ]:
# vector2d_v1.py
from array import array
import math

class Vector2d:
  typecode = 'd' # typecode is a class attr
                 # we'll use when converting Vector2d <=> bytes

  def __init__(self, x, y):
    self.x = float(x) # converting x to float catches errors early
    self.y = float(y)

  def __iter__(self):
    """
    __iter__ makes a Vector2d iterable
    """
    # equivalent to yield self.x; yield self.y
    return (i for i in (self.x, self.y))

  def __repr__(self):
    """
    __repr__ builds a string by interpolating the components
    with {!r} to get their repr; because Vector2d is iterable
    *self feeds the x and y component to format
    """
    class_name = type(self).__name__
    return '{}({!r}, {!r})'.format(class_name, *self)

  def __str__(self):
    return str(tuple(self))

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) +
            bytes(array(self.typecode, self)))

  # classmethod decorator modifies a method
  # so it can be called directly on a class
  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(*memv)

  def __eq__(self, other):
    """
    To quickly compare all components,
    build tuples out of the operands
    But there is a side effect:
    e.g. Vector(3, 4) == [3, 4]

    """
    return tuple(self) == tuple(other)

  def __abs__(self):
    return math.hypot(self.x, self.y)

  def __bool__(self):
    return bool(abs(self))

  def __format__(self, fmt_spec=''):
    components = (format(c, fmt_spec) for c in self)
    return '({}, {})'.format(*components)

## `classmethod` vs `staticmethod`

`classmethod` changes the way the method is called, so it receives the class itself as the first argument, instead of an instance. Its most common use is for alternative constructors, like `frombytes`

In contrast, `staticmethod` decorators changes a method so that it receives no special first argument. In essence, a static method is just like a plain function that happens to live in a class body, instaed of being defined at the module level.

In [ ]:
class Demo:
  @classmethod
  def klassmeth(*args):
    return args

  @staticmethod
  def statmeth(*args):
    return args

In [ ]:
Demo.klassmeth()
Demo.klassmeth('Spam')

(__main__.Demo, 'Spam')

In [ ]:
Demo.statmeth()
Demo.statmeth('Spam')

('Spam',)

# Formatted Displays

In [ ]:
brl = 1 / 4.82

In [ ]:
brl

0.20746887966804978

In [ ]:
format(brl, '0.4f') # built-in format fcn

'0.2075'

In [ ]:
'1 BRL = {rate:0.2f} USD'.format(rate=brl)

'1 BRL = 0.21 USD'

In [ ]:
f'1 USD = {1 / brl:0.2f} BRL'

'1 USD = 4.82 BRL'

`int` type supports 'b' and 'x' for base 2 and base 16

In [ ]:
format(42, 'b')
format(42, 'x')

'2a'

In [ ]:
format(2 / 3, '.2%') # percentage display

'66.67%'

Format Specification Mini-Language is extensible because each class gets to interpret the `format_spec` argument as it likes.

In [ ]:
from datetime import datetime

now = datetime.now()
format(now, '%H:%M:%S')
"It's now {:%I:%M %p}".format(now)

"It's now 02:50 AM"

In [ ]:
v1 = Vector2d(3, 4)

If a class has no `__format__`, the method inherited from `object` returns `str(my_object)`.

In [ ]:
format(v1)

'(3.0, 4.0)'

However, if you pass a format specifier, `object.__format__` raises TypeError

In [ ]:
format(v1, '.3f')

TypeError: unsupported format string passed to Vector2d.__format__

In [ ]:
v1 = Vector2d(3, 4)

In [ ]:
format(v1)

'(3.0, 4.0)'

In [ ]:
format(v1, '.3f')

'(3.000, 4.000)'

In [ ]:
format(v1, '.3e')

'(3.000e+00, 4.000e+00)'

Add custom fornatting: polar coordinate

In [ ]:
# vector2d_v1.py
from array import array
import math

class Vector2d:
  typecode = 'd' # typecode is a class attr
                 # we'll use when converting Vector2d <=> bytes

  def __init__(self, x, y):
    self.x = float(x) # converting x to float catches errors early
    self.y = float(y)

  def __iter__(self):
    """
    __iter__ makes a Vector2d iterable
    """
    # equivalent to yield self.x; yield self.y
    return (i for i in (self.x, self.y))

  def __repr__(self):
    """
    __repr__ builds a string by interpolating the components
    with {!r} to get their repr; because Vector2d is iterable
    *self feeds the x and y component to format
    """
    class_name = type(self).__name__
    return '{}({!r}, {!r})'.format(class_name, *self)

  def __str__(self):
    return str(tuple(self))

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) +
            bytes(array(self.typecode, self)))

  # classmethod decorator modifies a method
  # so it can be called directly on a class
  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(*memv)

  def __eq__(self, other):
    """
    To quickly compare all components,
    build tuples out of the operands
    But there is a side effect:
    e.g. Vector(3, 4) == [3, 4]

    """
    return tuple(self) == tuple(other)

  def __abs__(self):
    return math.hypot(self.x, self.y)

  def __bool__(self):
    return bool(abs(self))

  def angle(self):
    return math.atan2(self.y, self.x)

  def __format__(self, fmt_spec=''):
    if fmt_spec.endswith('p'):
      fmt_spec = fmt_spec[:-1]
      coords = (abs(self), self.angle())
      outer_fmt = '<{}, {}>'
    else:
      coords = self
      outer_fmt = '({}, {})'
    components = (format(c, fmt_spec) for c in coords)
    return outer_fmt.format(*components)



In [ ]:
format(Vector2d(1, 1), 'p')

'<1.4142135623730951, 0.7853981633974483>'

In [ ]:
format(Vector2d(1, 1), '.3ep')

'<1.414e+00, 7.854e-01>'

In [ ]:
format(Vector2d(1, 1), '0.5fp')

'<1.41421, 0.78540>'

## A Hashable Vector2d

In [ ]:
v1 = Vector2d(3, 4)

In [ ]:
hash(v1)

TypeError: unhashable type: 'Vector2d'

In [ ]:
set([v1])

TypeError: unhashable type: 'Vector2d'

To make a `Vector2d` hashable, we must implement `__hash__`

In [7]:
# vector2d_v3.py
from array import array
import math

class Vector2d:
  typecode = 'd' # typecode is a class attr
                 # we'll use when converting Vector2d <=> bytes

  __match_args__ = ('x', 'y') # class attribute listing the instance attributes in the order
                              # they will be used for positional pattern matching
  def __init__(self, x, y):
    self.__x = float(x) # converting x to float catches errors early
    self.__y = float(y)

  @property
  def x(self):
    return self.__x

  @property
  def y(self):
    return self.__y

  def __iter__(self):
    """
    __iter__ makes a Vector2d iterable
    """
    # equivalent to yield self.x; yield self.y
    return (i for i in (self.x, self.y))

  def __repr__(self):
    """
    __repr__ builds a string by interpolating the components
    with {!r} to get their repr; because Vector2d is iterable
    *self feeds the x and y component to format
    """
    class_name = type(self).__name__
    return '{}({!r}, {!r})'.format(class_name, *self)

  def __str__(self):
    return str(tuple(self))

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) +
            bytes(array(self.typecode, self)))

  # classmethod decorator modifies a method
  # so it can be called directly on a class
  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(*memv)

  def __eq__(self, other):
    """
    To quickly compare all components,
    build tuples out of the operands
    But there is a side effect:
    e.g. Vector(3, 4) == [3, 4]

    """
    return tuple(self) == tuple(other)

  def __abs__(self):
    return math.hypot(self.x, self.y)

  def __bool__(self):
    return bool(abs(self))

  def angle(self):
    return math.atan2(self.y, self.x)

  def __format__(self, fmt_spec=''):
    if fmt_spec.endswith('p'):
      fmt_spec = fmt_spec[:-1]
      coords = (abs(self), self.angle())
      outer_fmt = '<{}, {}>'
    else:
      coords = self
      outer_fmt = '({}, {})'
    components = (format(c, fmt_spec) for c in coords)
    return outer_fmt.format(*components)

  def __hash__(self):
    return hash((self.x, self.y))

In [ ]:
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)
hash(v1), hash(v2)

(1079245023883434373, 1994163070182233067)

In [ ]:
{v1, v2}

{Vector2d(3.0, 4.0), Vector2d(3.1, 4.2)}

13:57 -

## Supporting Positional Pattern Matching

In [8]:
def keyword_pattern_demo(v: Vector2d) -> None:
  match v:
    case Vector2d(0, 0):
      print(f'{v!r} is null')
    case Vector2d(0):
      print(f'{v!r} is vertical')
    case Vector2d(_, 0):
      print(f'{v!r} is horizontal')
    case Vector2d(x, y) if x==y:
      print(f'{v!r} is diagonal')
    case _:
      print(f'{v!r} is awesome')

In [13]:
keyword_pattern_demo(Vector2d(0, 3))

Vector2d(0.0, 3.0) is vertical


In [14]:
# Review

v1 = Vector2d(3, 4)
print(v1.x, v1.y)

3.0 4.0


In [15]:
x, y = v1
x, y

(3.0, 4.0)

In [16]:
v1

Vector2d(3.0, 4.0)

In [17]:
v1_clone = eval(repr(v1))

In [18]:
v1 == v1_clone

True

In [19]:
v1 is v1_clone

False

In [20]:
octets = bytes(v1)

In [21]:
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [22]:
abs(v1)

5.0

In [23]:
bool(v1), bool(Vector2d(0, 0))

(True, False)

In [24]:
v1_clone = Vector2d.frombytes(bytes(v1))

In [25]:
v1_clone

Vector2d(3.0, 4.0)

In [26]:
v1 == v1_clone

True

In [27]:
format(v1)

'(3.0, 4.0)'

In [28]:
format(v1, '.3e')

'(3.000e+00, 4.000e+00)'

In [29]:
format(v1, '.2f')

'(3.00, 4.00)'

In [30]:
v1.angle()

0.9272952180016122

In [31]:
Vector2d(1, 0).angle()

0.0

In [32]:
abs(Vector2d(0, 1).angle() - math.pi/2) < 10**-8

True

In [33]:
abs(Vector2d(1, 1).angle() - math.pi/4) < 10**-8

True

# Private and "Protected" Attributes in Python

If you name an instance attribute in the form `__mood`, Python stores the name in the instance `__dict__` prefixed with a leading underscore and the class name, `__mood` becomes `_Dog__mood`. This language feature goes by the lovely name of "name mangling"

In [34]:
v1 = Vector2d(3, 4)
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [35]:
v1._Vector2d__x = 4.0

In [36]:
v1

Vector2d(4.0, 4.0)

Name mangling is about safety, not security: it's designed to prevent accidental access and not malicious prying.

# Saving Memory with `__slots__`

By default, Python stores the attributes of each instance in a `dict` named `__dict__`. As we saw earlier, a `dict` has a significant memory overhead. But if you define a class attr named `__slots__` holding a sequence of attribute names, Python uses an alternative storage model for the instance attributes: the attributes named in `__slots__` are stored in a hidden array or references that use less memory than a `dict`.

In [38]:
class Pixel:
  __slots__ = ('x', 'y') # it must be present when this class is created; adding / changing later has no effect
                         # attribute names may be in a tuple or list

In [39]:
p = Pixel()

In [40]:
p.__dict__ # first effect: instances of Pixel have no __dict__

AttributeError: 'Pixel' object has no attribute '__dict__'

In [43]:
p.x = 10

In [44]:
p.y = 20

In [46]:
p.color = 'red' # Second effect: trying to set an attribute not listed in __slots__ raises Error

AttributeError: 'Pixel' object has no attribute 'color'

In [55]:
class OpenPixel(Pixel):
  pass

In [56]:
op = OpenPixel()

Suprise: instances of `OpenPixel` have a `__dict__`

In [57]:
op.__dict__

{}

In [58]:
op.x = 8

In [59]:
op.__dict__

{}

In [60]:
op.x

8

In [61]:
op.color = 'green'

In [63]:
op.__dict__

{'color': 'green'}

Conclusion: Effect of `__slots__` is only partially inheritied by a subclass

In [64]:
class ColorPixel(Pixel):
  __slots__ = ('color', )

In [65]:
cp = ColorPixel()
cp.__dict__

AttributeError: 'ColorPixel' object has no attribute '__dict__'

In [66]:
cp.x = 2
cp.color = 'blue'

In [67]:
cp.flavor = 'banana'

AttributeError: 'ColorPixel' object has no attribute 'flavor'

## Simple Measure of `__slots__` Savings

In [74]:
# vector2d_v3_slots.py
from array import array
import math

class Vector2d:
  __match_args__ = ('x', 'y') # class attribute listing the instance attributes in the order
                              # they will be used for positional pattern matching
  __slots__ = ('__x', '__y')

  typecode = 'd' # typecode is a class attr
                 # we'll use when converting Vector2d <=> bytes


  def __init__(self, x, y):
    self.__x = float(x) # converting x to float catches errors early
    self.__y = float(y)

  @property
  def x(self):
    return self.__x

  @property
  def y(self):
    return self.__y

  def __iter__(self):
    """
    __iter__ makes a Vector2d iterable
    """
    # equivalent to yield self.x; yield self.y
    return (i for i in (self.x, self.y))

  def __repr__(self):
    """
    __repr__ builds a string by interpolating the components
    with {!r} to get their repr; because Vector2d is iterable
    *self feeds the x and y component to format
    """
    class_name = type(self).__name__
    return '{}({!r}, {!r})'.format(class_name, *self)

  def __str__(self):
    return str(tuple(self))

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) +
            bytes(array(self.typecode, self)))

  # classmethod decorator modifies a method
  # so it can be called directly on a class
  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(*memv)

  def __eq__(self, other):
    """
    To quickly compare all components,
    build tuples out of the operands
    But there is a side effect:
    e.g. Vector(3, 4) == [3, 4]

    """
    return tuple(self) == tuple(other)

  def __abs__(self):
    return math.hypot(self.x, self.y)

  def __bool__(self):
    return bool(abs(self))

  def angle(self):
    return math.atan2(self.y, self.x)

  def __format__(self, fmt_spec=''):
    if fmt_spec.endswith('p'):
      fmt_spec = fmt_spec[:-1]
      coords = (abs(self), self.angle())
      outer_fmt = '<{}, {}>'
    else:
      coords = self
      outer_fmt = '({}, {})'
    components = (format(c, fmt_spec) for c in coords)
    return outer_fmt.format(*components)

  def __hash__(self):
    return hash((self.x, self.y))

In [75]:
v1 = Vector2d(1.1, 2.2)
dumpd = bytes(v1)

In [76]:
dumpd

b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'

In [77]:
len(dumpd)

17

In [78]:
v1.typecode = 'f'

AttributeError: 'Vector2d' object attribute 'typecode' is read-only

In [79]:
class ShortVector2d(Vector2d):
  typecode = 'f'

In [80]:
sv = ShortVector2d(1/11, 1/27)

In [81]:
sv

ShortVector2d(0.09090909090909091, 0.037037037037037035)

In [82]:
len(bytes(sv))

9